## Building an Agentic Application with Dynamically Generated Tool Calling
 
Function or tool calling is a powerful feature of LLMs that allows the LLM to call a function with arguments. This is a powerful way to extend the capabilities of LLMs to perform complex tasks. Typically, the function is defined by the developer and passed to the LLM as part of the prompt. The developer needs to pre-define the function, its arguments, and the expected output. 


In context of this Cookbook, **Dynamically Generated Tool is a function or code block that is generated by the LLM itself at runtime based on the user's prompt**.  This is a powerful way to extend the capabilities of LLMs to perform complex tasks, where the developer does not need to pre-define the function thus constraining the LLM's ability to set of pre-defined scenarios. 

In this paradigm, **Dynamically Generated Tool Calling is giving the LLM ability to dynamically generate and execute a code block at runtime.** 

This Cookbook demonstrates how to implement an “agentic” application — one that can generate and execute a tool call in pursuit of a specified goal at runtime. Large Language Models (LLMs) such as OpenAI o1 can generate sophisticated code, making them highly valuable for tasks such as AI assisted programming. We'll use this capability to generate function code for dynamic tools that can be executed in an isolated sandbox environment. 

Such an dynamic approach can be applied to a wide array of tasks, including: Data analysis and Visualization, Data manipulation and Transformation, Generating and Execution Machine Learning workloads, Process automation and Scription, ... and many more that will emerge as we experiment with this dynamic tool calling framework. 

**Following this Cookbook, you will learn:**  
1.	Set up an isolated Python code execution environment using Docker
2.	Configure your own code interpreter tool for LLM agents
3.	Setup a clear separation of Agentic concerns for security and safety
4.	Orchestrate agents to efficiently accomplish a given task 
5.	Agentic application that can dynamically generate and execute code

**Prerequisites:** 
Review the [Object-Oriented Approach to Designing Agentic LLM Solutions](./Object_oriented_approach_to_designing_agentic_LLM_solutions.ipynb) Cookbook to understand the core classes and principles for designing Agentic LLM solutions with a focus on Object-Oriented Programming (OOP). 

#### ⚠️ A WORD OF CAUTION:        
##### LLMs could generate harmful code with unintended consequences. As a best practice, isolate the code execution environment with only required read-only access to resources as needed by the task. This Cookbook will demonstrate an option to accomplish this goal using Docker to create a sandbox environment for code execution. **Do not auto execute LLM generated code on your host machine.** 

Let's consider a scenario where given a set of data, we want the LLM to answer a set of question. The data is in the form of a CSV files ![AAPL.csv](./resources/data/AAPL.csv) containing Apple's financial data with thousands of rows, and ![AAPL_2024.csv](./resources/data/AAPL_2024.csv) containing Apple's financial data for the last 15 years. Here are some questions we want the LLM to answer:
1. What was Apple's year on year Revenue growth from 2009 to 2024? 
2. What was Apple's closing price on a given date? 
3. What was Apple's total return for a given year? 
4. Plot the closing price of Apple over time. 

Using the traditional **Static Tool Calling** approach, we would need to pre-define the function for each of these questions. This would limit the LLM's ability answer any other questions not defined in the pre-defined set of functions. We overcome this limitation by using the **Dynamic Tool Calling** approach where the LLM generates and executes a function at runtime based on the user's prompt. 

## Overview
Let's dive into the steps to build the Agentic Applicaiton with Dynamic Tool Calling. There are three components to this application:

#### Step 1: Set up an isolated Python code execution environment

We need a secure environment where our LLM generated function calls can be executed. Given the word of caution, we want to avoid directly running the LLM generated code on the host machine. We will use a Docker container environment with restricted resource access (e.g., no network access). By default, Docker containers cannot access the host machine’s file system, which helps ensure that any code generated by the LLM remains contained. 

Within this container, we will preconfigure the necessary tools and libraries for data analysis and visualization—namely Python and commonly used Python libraries. This container can also be setup for task specific requirements such as access the an internal database for read-only access, or machine learning libraries such as Pyspark, Pytorch, Tensorflow, etc. 

#### Step 2: Define and Test the Agents

Let's fitst answer the question "**What is an Agent?**" before we define our agents. In the context of this Cookbook, an Agent is:
1. Set of instructions for the LLM to follow, i.e. the developer prompt
2. A LLM model, and ability to call the model via the API 
3. Tool call access to a function, and ability to call the function 

For our purposes, we will define two agents. 
1.	**Agent 1: File Access Agent (with Static Tool Calling)**
- Instructions to understand the contents of the file to provide as context to Agent 2.
- Has access to the host machine’s file system. 
- Can read a file from the host and copy it into the Docker container.
- Cannot access the code interpreter tool. 

2.	**Agent 2: Python Code Generator and Executor (with Dynamically Generated Tool Calling)**
- Recieve the file content's context from Agent 1.
- Instructions to generate a Python script to answer the user's question.
- Has access to the code interpreter within the Docker container, which is used to execute Python code.
- Has access only to the file system inside the Docker container (not the host).
- Cannot access the host machine’s file system or the network.

This separation concerns of the File Access (Agent 1) and the Code Generator and Executor (Agent 2) is crucial to prevent the LLM from directly accessing or modifying the host machine. **Limit the Agent 1 to Static Tool Calling as it has access to the host file system.**

| Agent | Type of Tool Call | Access to Host File System | Access to Docker Container File System | Access to Code Interpreter |
|-------|-------------------|----------------------------|----------------------------------------|----------------------------|
| Agent 1: File Access | Static or Pre-defined | Yes | Yes | No |
| Agent 2: Python Code Generator and Executor | Dynamic | No | Yes | Yes |


#### Step 3: Set up Agentic Orchestration to run the application 
There are various ways to orchestrate the Agents based on the application requirements. In this example, we will use a simple orchestration where the user provides a task and the agents are called in sequence to accomplish the task. The overall orchestration is shown below:

![Agentic Workflow Orchestration](./resources/images/AgenticWorkflow.png)



## Let's get started


### Prerequisites
Before you begin, ensure you have the following installed and configured on your host machine:

1. Docker: installed and running on your local machine. You can learn more about Docker and [install it from here](https://www.docker.com/). 
2. Python: installed on your local machine. You can learn more about Python and [install it from here](https://www.python.org/downloads/). 
3. OpenAI API key: set up on your local machine as an environment variable. You can learn more about OpenAI API key and [set it up from here](https://platform.openai.com/docs/api-reference/introduction). 


### Step 1: Set up an Isolated Python Code Execution Environment 

Lets define a Dockerized container environment that will be used to execute our code. I have defined the dockerfile in the directory `resources/docker/dockerfile` that will be used to create the container environment with the following specifications:
- Python 3.10 as the base 
- A non-root user 
- Preinstall the packages in requirements.txt  

Contents of the dockerfile:

```dockerfile
# Use Python 3.10 as the base image
FROM python:3.10

RUN apt-get update && \
    apt-get install -y build-essential && \
    rm -rf /var/lib/apt/lists/*

# Create a non-root user
RUN useradd -m sandboxuser
USER sandboxuser
WORKDIR /home/sandboxuser

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

CMD ["python", "--version"]
```
The requirements.txt file contains all the potential packages our LLM generated code may need to accomplish its tasks. Given we will restrict the container from network access, so we need to pre-install the packages that are required for the task. Our LLM will not be allowed to install any additional packages for security purposes. 

This file is copied into the docker container and the packages are installed:

```
numpy
pandas
matplotlib
seaborn
```

Let's build the docker image with the following command. For the sake of brevity, I have redirected the output to grep the success message and print a message if the build fails.

In [ ]:
!docker build -t python_sandbox:latest ./resources/docker 2>&1 | grep -E "View build details|ERROR" || echo "Build failed."

Let's run the container in restricted mode. The container will run in the background. This is our opportunity to define the security policies for the container. It is good practice to only allow the bare minimum features to the container that are required for the task. By default, the container cannot access the host file system from within the container. Let's also restrict its access to network so it cannot access the Internet or any other network resources. 

In [ ]:
# Run the container in restricted mode. The container will run in the background.
!docker run -d --name sandbox --network none --cap-drop all --pids-limit 64 --tmpfs /tmp:rw,size=64M   python_sandbox:latest sleep infinity

Let's make sure container is running using the `docker ps` that should list our container. 

In [ ]:
!docker ps 

### Step 2: Define and Test the Agents

As defined in the overview, we will create two agents. 
1. File Access Agent (with Static Tool Calling)
2. Python Code Generator and Executor (with Dynamic Tool Calling)

Let's define a set of core classes that will be used to create the two agents.

BaseAgent: We start with an abstract base class that enforces a task method signature. This ensures that all concrete agents will implement task consistently.
ChatMessages: A class to store the conversation history.
ToolManager: A class to manage the tools that an agent can call.
ToolInterface: An abstract class for any 'tool' that an agent can call.

These classes are defined in the `resources/core_classes` directory. Let's add the parent directory to the Python path so we can import these classes in our notebook. 

In [ ]:
import sys, os

# Add the parent directory of 'core_classes' to the Python path
sys.path.append(os.path.abspath('resources/object_oriented_agents/core_classes'))


**Defining FileAccessAgent**

Let's start with definin the FileAccessTool that the FileAccessAgent will use. 

In [6]:
import subprocess
import pandas as pd
from typing import Dict, Any

from examples.agentic_solutions.resources.object_oriented_agents.core_classes.tool_interface import ToolInterface

class FileAccessTool(ToolInterface):
    """
    A tool to read CSV files and copy them to a Docker container.
    """

    def get_definition(self) -> Dict[str, Any]:
        return {
            "function": {
                "name": "safe_file_access",
                "description": (
                    "Read the contents of a file in a secure manner "
                    "and transfer it to the Python code interpreter docker container"
                ),
                "parameters": {
                    "type": "object",
                    "properties": {
                        "filename": {
                            "type": "string",
                            "description": "Name of the file to read"
                        }
                    },
                    "required": ["filename"]
                }
            }
        }

    def run(self, arguments: Dict[str, Any]) -> str:
        filename = arguments["filename"]
        return self.safe_file_access(filename)

    def safe_file_access(self, filename: str) -> str:
        if not filename.endswith('.csv'):
            return "Error: The file is not a CSV file."

        # Ensure the path is correct
        if not os.path.dirname(filename):
            filename = os.path.join('./resources/data', filename)

        try:
            df = pd.read_csv(filename)
            copy_output = self.copy_file_to_container(filename)
            head_str = df.head(6).to_string()
            return f"{copy_output}\nThe file content for the first 6 rows is:\n{head_str}"
        except FileNotFoundError:
            return f"Error: The file '{filename}' was not found."
        except Exception as e:
            return f"Error while reading the CSV file: {str(e)}"

    @staticmethod
    def copy_file_to_container(local_file_name: str, container_name: str = "sandbox") -> str:
        container_home_path = "/home/sandboxuser"

        if not os.path.isfile(local_file_name):
            raise FileNotFoundError(f"The local file '{local_file_name}' does not exist.")

        # Check if container is running
        check_container_cmd = ["docker", "inspect", "-f", "{{.State.Running}}", container_name]
        result = subprocess.run(check_container_cmd, capture_output=True, text=True)
        if result.returncode != 0 or result.stdout.strip() != "true":
            raise RuntimeError(f"The container '{container_name}' is not running.")

        # Copy the file into the container
        container_path = f"{container_name}:{container_home_path}/{os.path.basename(local_file_name)}"
        subprocess.run(["docker", "cp", local_file_name, container_path], check=True)

        # Verify the file was copied
        verify_cmd = ["docker", "exec", container_name, "test", "-f",
                      f"{container_home_path}/{os.path.basename(local_file_name)}"]
        verify_result = subprocess.run(verify_cmd, capture_output=True, text=True)
        if verify_result.returncode != 0:
            raise RuntimeError(f"Failed to verify the file '{local_file_name}' in the container '{container_name}'.")

        return f"Copied {local_file_name} into {container_name}:{container_home_path}/."

Now, let's define the FileAccessAgent. 

In [7]:
# agents/file_access_agent.py
from examples.agentic_solutions.resources.object_oriented_agents.core_classes.base_agent import BaseAgent
from examples.agentic_solutions.resources.object_oriented_agents.core_classes.tool_manager import ToolManager

class FileAccessAgent(BaseAgent):
    """
    Agent that can only use the 'safe_file_access' tool to read CSV files.
    """

    def __init__(self, developer_prompt: str, model_name: str = "gpt-4o"):
        super().__init__(developer_prompt, model_name)
        self.setup_tools()

    def setup_tools(self) -> None:
        self.tool_manager = ToolManager()
        # Register the one tool this agent is allowed to use
        self.tool_manager.register_tool(FileAccessTool())

Let's define the FileAccessAgent. We'll define the tool definition and the tool call handler within the class to keep the code organized and coupled. 


Let's test this agent. We'll use Apple 2009-2024.csv file in the resources/data directory. Credits: [Apple Financials 2009-2024](https://www.kaggle.com/datasets/jamiedcollins/hjsjdjdjdjd?resource=download)

In [8]:
file_ingestion_prompt = """
You are a helpful data science assistant. The user will provide the name of a CSV file that contains relational data. The file is in the directory ./resources/data

Instructions:
1. When the user provides the CSV file name, use the 'safe_read_file' tool to read and display the first 6 lines of that file.
2. If the specified file does not exist in the provided directory, return an appropriate error message (e.g., "Error: The specified file does not exist in the provided directory").
3. The user may request data analysis based on the file’s contents, but you should NOT perform or write code for any data analysis. Your only task is to read and return the first 6 lines of the file.

Do not include any additional commentary or code not related to reading the file.
"""  

file_ingestion_agent = FileAccessAgent(file_ingestion_prompt)

file_ingestion_agent_output = file_ingestion_agent.task("Read the file Apple 2009-2024.csv")

print(file_ingestion_agent_output)

TypeError: 'ChatCompletionMessageToolCall' object is not subscriptable

Define the second agent that can generate and execute code. 

In [63]:
class PythonExecAgent(BaseAgent):
    """
    An agent that can only call 'execute_python_code' to run Python code in a container.
    """
    def get_tool_definition(self) -> Dict[str, Any]:
        """
        Returns the single tool (function) this agent can call.
        """
        return ({
                    "type": "function",
                    "function": {
                        "name": "execute_python_code",
                        "description": "Executes Python code securely in a container",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "python_code": {
                                    "type": "string",
                                    "description": "The Python code to execute"
                                }
                            },
                            "required": ["python_code"]
                        }
                    }
                })
    
    def handle_tool_call(self, tool_arguments: Dict[str, Any]) -> str:
        """
        Handles the call to 'execute_python_code'.
        """
        python_code = tool_arguments["python_code"]
        return self.execute_python_code(python_code)

    def execute_python_code(self, code: str) -> str:
        """
        Run Python code in a container and return the output.
        """
        print("[PythonExecAgent] Executing Python code in a container:")
        code = code.strip('"""')
        print(code)
        # Function that actually runs code in a container
        output, errors = self.run_code_in_container(code)
        
        if errors:
            return f"[Error]\n{errors}"
        
        return output

    # Helper function to run code in the container 
    @staticmethod
    def run_code_in_container(code: str, container_name: str = "sandbox") -> Tuple[str, str]:
        # The command we run inside the container uses `python -c` and reads code from stdin.
        # `exec(sys.stdin.read())` executes all code passed via stdin.
        cmd = [
            "docker", "exec", "-i", 
            container_name, 
            "python", "-c", "import sys; exec(sys.stdin.read())"
        ]
        
        # Run the command, providing the code via stdin
        process = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        out, err = process.communicate(code)

        return out, err

In [ ]:
data_analysis_prompt = """  
You are a helpful data science assistant. The user will provide:
	1.	The name of a CSV file containing relational data in a table.
	2.	The first 6 lines of the CSV (headers plus sample rows) to clarify column names and data types.
	3.	A question or a task about the data.

Your task is to generate a Python script that:
	1.	Answers the user’s question or task by analyzing the DataFrame.
	2.	Checks if the specified file exists in the current directory before reading. If it does not exist, return an error message:
"Error: The specified file does not exist in the current directory".
	3.	Handles potential mismatches in column names or data types.
	4.	Uses only Python standard libraries plus pandas, numpy, matplotlib, and seaborn.
	5.	Generate only the Python code in a single code block (no explanatory text or comments). 
    6.  Call the tool execute_python_code that will execute the code inside a docker container. 
    7.  Because the code is executed inside a container there is no display attached to the output. 
    8.  Always return the output from the code execution. 
    9.  If the code generates an image or a plot, print the plot as base64 encoded image to stdout. sys.stdout.write(encoded_image). Don't use any extraneous text.
	10.	Put appropriate error handling such as the column names and data types are not as expected.

Files if specified in the prompt are in the directory /home/sandboxuser
"""

data_analysis_agent = PythonExecAgent(data_analysis_prompt)

data_analysis_agent_output = data_analysis_agent.task(f"""What was Apple's revenue in 2024?\n
                                                      {file_ingestion_agent_output}""")

print(data_analysis_agent_output)

### Step 3: Set up Agentic Orchestration to run the application 

Let's get a data set with thousands of rows that couldn't possibly fit in the context window of the LLM, and see if our agentic orchestration can handle it with the help of the code interpreter. We will usee APPL.csv file in the resources/data directory. Credits: [Kaggle.com Apple Stock Data](https://www.kaggle.com/datasets/varpit94/apple-stock-data-updated-till-22jun2021)


In [ ]:

import base64
import binascii

def is_base64(s):
    try:
        base64.b64decode(s, validate=True)
        return True
    except binascii.Error:
        return False

while True:
    user_input = input("Please enter your question or task. Type 'exit' to stop.     ")
    if user_input == "exit":
        print("Exiting the application.")
        break
    question = user_input

    print("------------------- Question or task -------------------")
    print(question)

    file_ingestion_agent_output = file_ingestion_agent.task(question)

    print("------------------- Step 1 Understanding the contents of the file -------------------")
    print(file_ingestion_agent_output)

    data_analysis_agent = PythonExecAgent(data_analysis_prompt)

    print("------------------- Step 2 Generating the Python script and executing it in a container -------------------")
    data_analysis_agent_output = data_analysis_agent.task(f"""{question} + "\n" + {file_ingestion_agent_output}""")

    print("------------------- Step 3 Displaying the output -------------------")
    if is_base64(data_analysis_agent_output):
        # decode image
        decoded_image = base64.b64decode(data_analysis_agent_output)
        display(Image(data=decoded_image))
    else:
        # handle text
        print(data_analysis_agent_output)


